<a href="https://colab.research.google.com/github/shwethashaji-ai/cognifyzTech_ML/blob/main/restaurant_recom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


In [5]:
class RestaurantRecommender:
    def __init__(self, file_path):
        #Initialize and preprocess dataset
        self.df = self.load_and_clean_data(file_path)
        self.similarity_matrix = self.compute_similarity()

    def load_and_clean_data(self, file_path):
        """Loads the dataset and performs preprocessing."""
        df = pd.read_csv('/content/Dataset .csv')



In [6]:
df = pd.read_csv('/content/Dataset .csv')
df


,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9546,5915730,Naml۱ Gurme,208,��stanbul,"Kemanke�� Karamustafa Pa��a Mahallesi, R۱ht۱m ...",Karak�_y,"Karak�_y, ��stanbul",28.977392,41.022793,Turkish,...,Turkish Lira(TL),No,No,No,No,3,4.1,Green,Very Good,788
9547,5908749,Ceviz A��ac۱,208,��stanbul,"Ko��uyolu Mahallesi, Muhittin ��st�_nda�� Cadd...",Ko��uyolu,"Ko��uyolu, ��stanbul",29.041297,41.009847,"World Cuisine, Patisserie, Cafe",...,Turkish Lira(TL),No,No,No,No,3,4.2,Green,Very Good,1034
9548,5915807,Huqqa,208,��stanbul,"Kuru�_e��me Mahallesi, Muallim Naci Caddesi, N...",Kuru�_e��me,"Kuru�_e��me, ��stanbul",29.034640,41.055817,"Italian, World Cuisine",...,Turkish Lira(TL),No,No,No,No,4,3.7,Yellow,Good,661
9549,5916112,A���k Kahve,208,��stanbul,"Kuru�_e��me Mahallesi, Muallim Naci Caddesi, N...",Kuru�_e��me,"Kuru�_e��me, ��stanbul",29.036019,41.057979,Restaurant Cafe,...,Turkish Lira(TL),No,No,No,No,4,4.0,Green,Very Good,901


In [7]:
df.head(5)

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


In [20]:
 # Fill missing cuisine values
df['Cuisines'] = df['Cuisines'].fillna('NAN')
# Display rows where 'Cuisines' is 'NAN'
display(df[df['Cuisines'] == 'NAN'])

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
84,17284105,Cookie Shoppe,216,Albany,"115 N Jackson St, Albany, GA 31701",Albany,"Albany, Albany",-84.154000,31.577200,NAN,...,Dollar($),NaN,NaN,NaN,No,1,3.4,Orange,Average,34
87,17284211,Pearly's Famous Country Cookng,216,Albany,"814 N Slappey Blvd, Albany, GA 31701",Albany,"Albany, Albany",-84.175900,31.588200,NAN,...,Dollar($),NaN,NaN,NaN,No,1,3.4,Orange,Average,36
94,17284158,Jimmie's Hot Dogs,216,Albany,"204 S Jackson St, Albany, GA 31701",Albany,"Albany, Albany",-84.153400,31.575100,NAN,...,Dollar($),NaN,NaN,NaN,No,1,3.9,Yellow,Good,160
297,17374552,Corkscrew Cafe,216,Gainesville,"51 W Main St, Dahlonega, GA 30533",Dahlonega,"Dahlonega, Gainesville",-83.985800,34.531800,NAN,...,Dollar($),NaN,NaN,NaN,No,3,3.9,Yellow,Good,209
328,17501439,Dovetail,216,Macon,"543 Cherry St, Macon, GA 31201",Macon,"Macon, Macon",-83.627979,32.836410,NAN,...,Dollar($),NaN,NaN,NaN,No,3,3.8,Yellow,Good,102
346,17606621,HI Lite Bar & Lounge,216,Miller,"109 N Broadway Ave, Miller, SD 57362",Miller,"Miller, Miller",-98.989100,44.515800,NAN,...,Dollar($),NaN,NaN,NaN,No,1,3.4,Orange,Average,11
368,17059060,Hillstone,216,Orlando,"215 South Orlando Avenue, Winter Park, FL 32789",Winter Park,"Winter Park, Orlando",-81.365260,28.596682,NAN,...,Dollar($),NaN,NaN,NaN,No,3,4.4,Green,Very Good,1158
418,17142698,Leonard's Bakery,216,Rest of Hawaii,"933 Kapahulu Ave, Honolulu, HI 96816",Kaimuki,"Kaimuki, Rest of Hawaii",-157.813432,21.284586,NAN,...,Dollar($),NaN,NaN,NaN,No,1,4.7,Dark Green,Excellent,707
455,17616465,Tybee Island Social Club,216,Savannah,"1311 Butler Ave, Tybee Island, GA 31328",Tybee Island,"Tybee Island, Savannah",-80.848297,31.995810,NAN,...,Dollar($),NaN,NaN,NaN,No,1,3.9,Yellow,Good,309


In [13]:
# Convert categorical Yes/No values into binary 1/0
yes_no_cols = ['Has Table booking', 'Has Online delivery', 'Is delivering now']
for col in yes_no_cols:
  df[col] = df[col].map({'Yes': 1, 'No': 0})

In [15]:
def select_features(df):
  """Selects necessary features from the DataFrame."""
  return df[['Restaurant Name', 'City', 'Cuisines', 'Average Cost for two', 'Price range', 'Aggregate rating', 'Votes',
            'Has Table booking', 'Has Online delivery']]

In [16]:
# Assuming 'df' is your original DataFrame
selected_features_df = select_features(df)
selected_features_df.head() # Display the first few rows of the new DataFrame

,Restaurant Name,City,Cuisines,Average Cost for two,Price range,Aggregate rating,Votes,Has Table booking,Has Online delivery
0,Le Petit Souffle,Makati City,"French, Japanese, Desserts",1100,3,4.8,314,NaN,NaN
1,Izakaya Kikufuji,Makati City,Japanese,1200,3,4.5,591,NaN,NaN
2,Heat - Edsa Shangri-La,Mandaluyong City,"Seafood, Asian, Filipino, Indian",4000,4,4.4,270,NaN,NaN
3,Ooma,Mandaluyong City,"Japanese, Sushi",1500,4,4.9,365,NaN,NaN
4,Sambo Kojin,Mandaluyong City,"Japanese, Korean",1500,4,4.8,229,NaN,NaN


In [48]:

class RestaurantRecommender:
    def __init__(self, df):
        self.df = df
        # Handle missing values before computing similarity
        self.df['Cuisines'] = self.df['Cuisines'].fillna('NAN')
        self.similarity_matrix = self.compute_similarity()

    def compute_similarity(self):
        """Creates a similarity matrix based on cuisines."""
        vectorizer = TfidfVectorizer(stop_words='english')
        cuisine_matrix = vectorizer.fit_transform(self.df['Cuisines'])
        return linear_kernel(cuisine_matrix, cuisine_matrix)

    def recommend(self, restaurant_name, top_n=5):
        """Finds and returns similar restaurants."""
        try:
            idx = self.df[self.df['Restaurant Name'] == restaurant_name].index[0]
            scores = list(enumerate(self.similarity_matrix[idx]))
            scores = sorted(scores, key=lambda x: x[1], reverse=True)[1:top_n + 1]
            indices = [i[0] for i in scores]
            return self.df.iloc[indices][['Restaurant Name', 'Cuisines', 'Aggregate rating']]
        except IndexError:
            return f"'{restaurant_name}' not found in the database. Try another name."

# Example usage
if __name__ == "__main__":
    df = pd.read_csv("/content/Dataset .csv")  # Load your DataFrame here
    recommender = RestaurantRecommender(df)
    restaurant_name = "Le Petit Souffle"  # Change for testing
    recommendations = recommender.recommend(restaurant_name)

    print("\nTop Recommended Restaurants:")
    print(recommendations)


Top Recommended Restaurants:
            Restaurant Name          Cuisines  Aggregate rating
4993        Tokyo Mon Amour  Japanese, French               3.1
70        Paris 6 Classique            French               3.4
259                  Django            French               4.3
460                    Jaan            French               3.8
461   Rhubarb Le Restaurant            French               3.9
